# 🔮 CardioFusion: Interactive Prediction Widget

## 📋 Overview
This notebook provides an **interactive prediction interface** using Jupyter widgets. Enter patient information and get real-time cardiovascular disease risk predictions.

### ✨ Features
- 🎨 Interactive form controls
- 🔄 Real-time predictions
- 📊 Visual risk assessment
- 🔍 SHAP explainability
- 💡 Clinical recommendations

---

In [ ]:
# Import libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Wedge
import warnings
warnings.filterwarnings('ignore')

# Import utility modules
from utils.model_utils import ModelPredictor
from utils.data_validator import DataValidator
from utils.shap_explainer import SHAPExplainer

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🎨 Widget interface ready to use!")

## 🔄 Load Models

In [ ]:
print("📂 Loading trained models...\n")

# Initialize model predictor
predictor = ModelPredictor('models')
success = predictor.load_models()

if success:
    print("\n✅ All models loaded successfully!")
    print(f"📊 Available models: {', '.join(predictor.get_available_models())}")
else:
    print("❌ Failed to load models. Please ensure models are trained.")

# Load background data for SHAP
try:
    train_data = pd.read_csv('train_data.csv')
    X_train = train_data.drop('Heart_Disease', axis=1)
    background_data = X_train.sample(min(100, len(X_train)), random_state=42)
    print("✅ Background data loaded for SHAP analysis")
except:
    background_data = None
    print("⚠️ Background data not available - SHAP analysis will be limited")

## 🎨 Create Interactive Form

In [ ]:
# Styled HTML header
header_html = """
<div style="background: linear-gradient(135deg, #1e3a8a 0%, #3b82f6 100%);
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h1 style="color: white; margin: 0; font-size: 2.5em;">🩺 CardioFusion</h1>
    <p style="color: #e0e7ff; margin: 10px 0 0 0; font-size: 1.2em;">
        Interactive Cardiovascular Risk Assessment
    </p>
</div>
"""
display(HTML(header_html))

# ============================================
# SECTION 1: Demographics
# ============================================

display(HTML('<h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">👤 Demographic Information</h2>'))

age_categories = ['18-24', '25-29', '30-34', '35-39', '40-44', '45-49',
                 '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+']

age_widget = widgets.Dropdown(
    options=age_categories,
    value='50-54',
    description='Age Group:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px')
)

sex_widget = widgets.RadioButtons(
    options=['Male', 'Female'],
    description='Sex:',
    style={'description_width': '150px'}
)

demo_box = widgets.HBox([age_widget, sex_widget])
display(demo_box)

# ============================================
# SECTION 2: Physical Measurements
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">📏 Physical Measurements</h2>'))

height_widget = widgets.IntSlider(
    value=170,
    min=100,
    max=250,
    step=1,
    description='Height (cm):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px'),
    readout=True
)

weight_widget = widgets.FloatSlider(
    value=75.0,
    min=30.0,
    max=300.0,
    step=0.5,
    description='Weight (kg):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px'),
    readout=True
)

bmi_display = widgets.HTML(value='<b>BMI:</b> 25.9')

def update_bmi(*args):
    height_m = height_widget.value / 100
    bmi = weight_widget.value / (height_m ** 2)
    
    # Color code BMI
    if bmi < 18.5:
        color = '#3b82f6'
        category = 'Underweight'
    elif bmi < 25:
        color = '#059669'
        category = 'Normal'
    elif bmi < 30:
        color = '#d97706'
        category = 'Overweight'
    else:
        color = '#dc2626'
        category = 'Obese'
    
    bmi_display.value = f'<div style="background: {color}; color: white; padding: 10px; border-radius: 5px; text-align: center; width: 200px;"><b>BMI: {bmi:.1f}</b><br><small>{category}</small></div>'

height_widget.observe(update_bmi, 'value')
weight_widget.observe(update_bmi, 'value')

display(height_widget)
display(weight_widget)
display(bmi_display)

# ============================================
# SECTION 3: Lifestyle Factors
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">🏃 Lifestyle Factors</h2>'))

exercise_widget = widgets.RadioButtons(
    options=['Yes', 'No'],
    value='Yes',
    description='Regular Exercise:',
    style={'description_width': '150px'}
)

smoking_widget = widgets.RadioButtons(
    options=['No', 'Yes'],
    value='No',
    description='Smoking History:',
    style={'description_width': '150px'}
)

lifestyle_box1 = widgets.HBox([exercise_widget, smoking_widget])
display(lifestyle_box1)

alcohol_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=30,
    description='Alcohol (units/mo):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

fruit_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=120,
    description='Fruit (servings/mo):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

vegetables_widget = widgets.IntSlider(
    value=12,
    min=0,
    max=128,
    description='Vegetables (servings/mo):',
    style={'description_width': '180px'},
    layout=widgets.Layout(width='500px')
)

fried_potato_widget = widgets.IntSlider(
    value=4,
    min=0,
    max=128,
    description='Fried Potato (servings/mo):',
    style={'description_width': '180px'},
    layout=widgets.Layout(width='500px')
)

display(alcohol_widget)
display(fruit_widget)
display(vegetables_widget)
display(fried_potato_widget)

# ============================================
# SECTION 4: Health Status
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">🏥 Health Status</h2>'))

general_health_widget = widgets.SelectionSlider(
    options=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'],
    value='Good',
    description='General Health:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

checkup_widget = widgets.Dropdown(
    options=['Within the past year', 'Within the past 2 years',
             'Within the past 5 years', '5 or more years ago', 'Never'],
    value='Within the past year',
    description='Last Checkup:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

diabetes_widget = widgets.Dropdown(
    options=['No', 'Yes', 'No, pre-diabetes or borderline diabetes',
             'Yes, but female told only during pregnancy'],
    value='No',
    description='Diabetes:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

display(general_health_widget)
display(checkup_widget)
display(diabetes_widget)

display(HTML('<br><h3>Medical Conditions:</h3>'))

depression_widget = widgets.Checkbox(value=False, description='Depression')
arthritis_widget = widgets.Checkbox(value=False, description='Arthritis')
skin_cancer_widget = widgets.Checkbox(value=False, description='Skin Cancer')
other_cancer_widget = widgets.Checkbox(value=False, description='Other Cancer')

conditions_box = widgets.HBox([
    widgets.VBox([depression_widget, arthritis_widget]),
    widgets.VBox([skin_cancer_widget, other_cancer_widget])
])
display(conditions_box)

# ============================================
# SECTION 5: View Mode Toggle
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">⚙️ Settings</h2>'))

view_mode_widget = widgets.RadioButtons(
    options=['Simple View', 'Detailed Analysis'],
    value='Simple View',
    description='Detail Level:',
    style={'description_width': '150px'}
)

display(view_mode_widget)

print("\n✅ Interactive form created!")
print("📝 Fill in the patient information above and run the next cell to get predictions.")

## 🔬 Generate Prediction

In [ ]:
# Collect patient data
patient_data = {
    'age_category': age_widget.value,
    'sex': sex_widget.value,
    'height_cm': height_widget.value,
    'weight_kg': weight_widget.value,
    'bmi': weight_widget.value / ((height_widget.value/100) ** 2),
    'exercise': exercise_widget.value,
    'smoking_history': smoking_widget.value,
    'alcohol_consumption': alcohol_widget.value,
    'fruit_consumption': fruit_widget.value,
    'green_vegetables_consumption': vegetables_widget.value,
    'fried_potato_consumption': fried_potato_widget.value,
    'general_health': general_health_widget.value,
    'checkup': checkup_widget.value,
    'diabetes': diabetes_widget.value,
    'depression': 'Yes' if depression_widget.value else 'No',
    'arthritis': 'Yes' if arthritis_widget.value else 'No',
    'skin_cancer': 'Yes' if skin_cancer_widget.value else 'No',
    'other_cancer': 'Yes' if other_cancer_widget.value else 'No'
}

# Validate and preprocess
validator = DataValidator()
is_valid, processed_data, errors, warnings = validator.validate_input(patient_data)

if not is_valid:
    print("❌ VALIDATION ERRORS:")
    for error in errors:
        print(f"  {error}")
else:
    if warnings:
        print("⚠️ WARNINGS:")
        for warning in warnings:
            print(f"  {warning}")
        print()
    
    # Preprocess
    input_df = validator.preprocess_for_model(patient_data)
    
    # Make prediction
    print("🔄 Analyzing patient data...\n")
    prediction = predictor.predict(input_df)
    
    # Get risk category
    risk_pct = prediction['risk_percentage']
    category, emoji, color = predictor.get_risk_category(risk_pct)
    
    # ============================================
    # DISPLAY RESULTS
    # ============================================
    
    # Header
    result_header = f"""
    <div style="background: linear-gradient(135deg, {color} 0%, {color}dd 100%);
                padding: 30px;
                border-radius: 10px;
                margin: 20px 0;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h1 style="color: white; margin: 0; font-size: 3em;">{emoji} {category}</h1>
        <h2 style="color: white; margin: 10px 0 0 0;">Risk Score: {risk_pct:.1f}%</h2>
    </div>
    """
    display(HTML(result_header))
    
    # Create visualization
    if view_mode_widget.value == 'Simple View':
        # Simple gauge chart
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Create gauge
        theta = np.linspace(0, np.pi, 100)
        
        # Background arcs
        colors_bg = ['#059669', '#f59e0b', '#d97706', '#dc2626']
        ranges = [(0, 30), (30, 50), (50, 70), (70, 100)]
        
        for i, (start, end) in enumerate(ranges):
            theta_range = theta[(theta >= np.pi * start / 100) & (theta <= np.pi * end / 100)]
            ax.fill_between(theta_range, 0, 0.8, color=colors_bg[i], alpha=0.3)
        
        # Needle
        needle_angle = np.pi * (1 - risk_pct / 100)
        ax.plot([needle_angle, needle_angle], [0, 0.7], 'k-', linewidth=3)
        ax.plot(needle_angle, 0.7, 'ko', markersize=10)
        
        # Labels
        ax.text(np.pi, -0.2, '0%', ha='center', fontsize=12, fontweight='bold')
        ax.text(0, -0.2, '100%', ha='center', fontsize=12, fontweight='bold')
        ax.text(np.pi/2, -0.4, f'{risk_pct:.1f}%', ha='center', fontsize=24,
                fontweight='bold', color=color)
        
        ax.set_xlim(0, np.pi)
        ax.set_ylim(-0.5, 1)
        ax.axis('off')
        ax.set_title('Cardiovascular Disease Risk Assessment', fontsize=16, fontweight='bold', pad=20)
        
        plt.tight_layout()
        plt.show()
        
        # Summary metrics
        summary_html = f"""
        <div style="display: flex; justify-content: space-around; margin: 20px 0;">
            <div style="text-align: center; padding: 20px; background: #f9fafb; border-radius: 10px; flex: 1; margin: 0 10px;">
                <h3 style="color: #64748b; margin: 0;">Prediction</h3>
                <p style="font-size: 24px; font-weight: bold; margin: 10px 0; color: {color};">{prediction['prediction_label']}</p>
            </div>
            <div style="text-align: center; padding: 20px; background: #f9fafb; border-radius: 10px; flex: 1; margin: 0 10px;">
                <h3 style="color: #64748b; margin: 0;">Confidence</h3>
                <p style="font-size: 24px; font-weight: bold; margin: 10px 0; color: {color};">{prediction['confidence']*100:.1f}%</p>
            </div>
            <div style="text-align: center; padding: 20px; background: #f9fafb; border-radius: 10px; flex: 1; margin: 0 10px;">
                <h3 style="color: #64748b; margin: 0;">Model</h3>
                <p style="font-size: 18px; font-weight: bold; margin: 10px 0; color: #1e3a8a;">Ensemble</p>
            </div>
        </div>
        """
        display(HTML(summary_html))
        
    else:
        # Detailed view with SHAP analysis
        print("\n📊 DETAILED ANALYSIS")
        print("=" * 50)
        
        # Individual model predictions
        if 'individual_models' in prediction:
            print("\n🤖 Individual Model Predictions:")
            for model_name, results in prediction['individual_models'].items():
                print(f"  {model_name:<25} {results['probability_disease']*100:>6.1f}% (weight: {results['weight']:.2f})")
        
        # SHAP Analysis
        if background_data is not None:
            try:
                print("\n🔍 Generating SHAP explanation...")
                model = list(predictor.models.values())[0]
                explainer = SHAPExplainer(model, background_data)
                explanation = explainer.explain_prediction(input_df)
                
                if 'error' not in explanation:
                    interpretation = explainer.generate_clinical_interpretation(explanation)
                    print(interpretation)
                    
                    # Recommendations
                    print("\n💡 CLINICAL RECOMMENDATIONS:")
                    recommendations = explainer.get_recommendations(explanation)
                    for i, rec in enumerate(recommendations, 1):
                        print(f"{i}. {rec}")
            except Exception as e:
                print(f"⚠️ SHAP analysis unavailable: {e}")
    
    # Disclaimer
    disclaimer_html = """
    <div style="background: #fef3c7; border-left: 4px solid #d97706; padding: 20px;
                border-radius: 8px; margin: 30px 0;">
        <h3 style="color: #92400e; margin-top: 0;">⚠️ Medical Disclaimer</h3>
        <p style="color: #92400e; margin-bottom: 0;">
            This tool is for <strong>educational and informational purposes only</strong>.
            It does not provide medical advice, diagnosis, or treatment. Always seek
            the advice of your physician or other qualified health provider with any
            questions you may have regarding a medical condition.
        </p>
    </div>
    """
    display(HTML(disclaimer_html))
    
    print("\n✅ Analysis complete!")

## 📝 Usage Instructions

### How to Use This Widget:

1. **Run all cells above** to create the interactive form
2. **Adjust the widgets** to enter patient information
3. **Select view mode**: Simple View or Detailed Analysis
4. **Run the prediction cell** to get results

### View Modes:

- **Simple View**: Quick risk assessment with gauge visualization
- **Detailed Analysis**: Complete analysis with SHAP explanations and recommendations

---

*CardioFusion - Professional ML for Heart Disease Prediction* 🩺